# Credits
Most of the contents were taken from [This Notebook](https://www.kaggle.com/drcapa/cassava-leaf-disease-classification-starter-keras)

# Intro
Welcome to the Cassava Leaf Disease Classification competition.

There are 5 classifications (click for further informations):
* 0: [Cassava Bacterial Blight (CBB)](https://en.wikipedia.org/wiki/Bacterial_blight_of_cassava)
* 1: [Cassava Brown Streak Disease (CBSD)](https://en.wikipedia.org/wiki/Cassava_brown_streak_virus_disease)
* 2: [Cassava Green Mottle (CGM)](https://en.wikipedia.org/wiki/Cassava_green_mottle_virus)
* 3: [Cassava Mosaic Disease (CMD)](https://en.wikipedia.org/wiki/Cassava_mosaic_virus)
* 4: Healthy"

We will give a simple starter notebook based on a CNN.

# Libraries

In [ ]:
%%capture
import sys
sys.path.append('/kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle')
! pip install -e /kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop,Adam

# Path

In [ ]:
path = '/kaggle/input/cassava-leaf-disease-classification/'
os.listdir(path)

# Functions

In [ ]:
def plot_bar(data, name):
    data_label = data[name].value_counts().sort_index()
    dict_train = dict(zip(data_label.keys(), ((data_label.sort_index())).tolist()))
    names = list(dict_train.keys())
    values = list(dict_train.values())
    plt.bar(names, values)
    plt.grid()
    plt.show()

# Load Data

In [ ]:
train_data = pd.read_csv(path+'train.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

# EDA

In [ ]:
print('number of train data:', len(train_data))
print('number of train images:', len(os.listdir(path+'train_images/')))
print('number of test images:', len(os.listdir(path+'test_images/')))

Distribution of the labels:

In [ ]:
plot_bar(train_data, 'label')

Plot an image:

In [ ]:
img = cv2.imread(path+'train_images/'+'1000015157.jpg')
plt.imshow(img)
plt.show()

# Prepare Data For Model

In [ ]:
batch_size = 4
img_size = 128
img_channel = 3
effnet = 0

## Train Labels And Class Weights

In [ ]:
# 
# class_weight = dict(zip(range(0, 5), (.value_counts().sort_index()/len(train_data))))
# class_weight

In [ ]:
import sklearn
y_train = to_categorical(train_data['label'])

weight = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', 
                                                classes = np.unique(train_data['label']),
                                                y = train_data['label'])
class_weight = dict(zip(range(0, 5), weight))
class_weight

## Image Data Generator

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, path, list_IDs, labels, batch_size, img_size, img_channel):
        self.path = path
        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_channel = img_channel
        self.indexes = np.arange(len(self.list_IDs))
        
    def __len__(self):
        return int(np.floor(len(self.list_IDs)/self.batch_size))
    
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    
    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, self.img_size, self.img_size, self.img_channel))
        y = np.empty((self.batch_size, 5), dtype=int)
        for i, ID in enumerate(list_IDs_temp):
            data_file = cv2.imread(self.path+ID)
            img = cv2.resize(data_file, (self.img_size, self.img_size))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            X[i, ] = img
            y[i, ] = self.labels[i]
        X = X.astype('float32')/255.0
#         X -= X.mean()
#         X /= X.std()
        return X, y

# Augmentation Pipeline (GPU)

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip(),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
#     tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor = 0.2, width_factor = 0.2),
    tf.keras.layers.experimental.preprocessing.RandomCrop(height = int(img_size*0.8), width = int(img_size*0.8)),
    tf.keras.layers.experimental.preprocessing.Resizing(height = int(img_size), width = int(img_size), interpolation = 'area'),   
])

# Plot Some Samples

In [ ]:
label2name = {"0": "Bacterial Blight",
"1": "Brown Streak Disease",
"2": "Green Mottle",
"3": "Mosaic Disease",
"4": "Healthy"}

In [ ]:
train_generator = DataGenerator(path+'train_images/', train_data['image_id'].tolist(), y_train, 8, img_size, img_channel)
X, y = train_generator.__getitem__(0)
y = np.max(y, axis = -1)

In [ ]:
def display(X, y):
    samples = X.shape[0]
    n_col = 4
    n_row = samples//4
    fig, ax  = plt.subplots(n_row, n_col, figsize = (n_col*5, n_row*5), constrained_layout = True)
    for row in range(n_row):
        for col in range(n_col):
            ax[row][col].imshow(X[row*n_col + col])
            ax[row][col].set_title(label2name[str(y[row*n_col + col])]+f'({str(y[row*n_col + col])})', fontsize = 15)
            ax[row][col].set_xticks([]);
            ax[row][col].set_yticks([]);
    plt.show()

## Before Augmentation

In [ ]:
display(X,y)

## After Augmentation

In [ ]:
display(data_augmentation(X),y)

# Callbacks

In [ ]:
def get_lr_callback(batch_size=8, show = False):
    lr_start   = 0.000000825
    lr_max     = 0.000000800 * 1 * batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr
    
    if show:
        plt.figure(figsize = (8, 5))
        plt.plot(np.arange(1, 12), [lrfn(x) for x in np.arange(1, 12)], marker = 'o')
        plt.xlabel('epoch')
        plt.ylabel('learaning_rate');
        plt.show()
        
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)
    return lr_callback


# Figure
get_lr_callback(batch_size=8, show = True)

# CheckPoint

In [ ]:
def get_ckpt():
    return tf.keras.callbacks.ModelCheckpoint('best.h5', 
                                        monitor='val_categorical_accuracy', 
                                        verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True,
                                        mode='max',
                                        save_freq='epoch')

# Define Model

In [ ]:
import efficientnet.tfkeras as efn   
model_dict = {0: efn.EfficientNetB0,
              1: efn.EfficientNetB1,
              2: efn.EfficientNetB2,
              3: efn.EfficientNetB3,
              4: efn.EfficientNetB4,
              5: efn.EfficientNetB5,
              6: efn.EfficientNetB6,
              7: efn.EfficientNetB7,}

def get_model():
    base_model = model_dict[effnet](input_shape=(img_size, img_size, img_channel),weights='imagenet',include_top=False)

    model = tf.keras.Sequential([
                                    data_augmentation,
                                    base_model,
                                    tf.keras.layers.GlobalAveragePooling2D(),
                                    tf.keras.layers.Dense(64, activation = 'relu'),
                                    tf.keras.layers.Dense(5, activation = 'softmax')
                                ])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model
# model.summary()

# Train Model

In [ ]:
from sklearn.model_selection import train_test_split
train_idx, val_idx = train_test_split(train_data.index, test_size = 0.2, stratify = train_data['label'], random_state = 42)
print(f'Train: {len(train_idx)}')
print(f'Val: {len(val_idx)}')
train_generator = DataGenerator(path+'train_images/', train_data['image_id'].loc[train_idx].tolist(), y_train[train_idx], batch_size, img_size, img_channel)
val_generator   = DataGenerator(path+'train_images/', train_data['image_id'].loc[val_idx].tolist(), y_train[val_idx], batch_size, img_size, img_channel)

In [ ]:
epochs = 12
model = get_model()
history = model.fit(train_generator,
                    steps_per_epoch = len(train_idx)//batch_size,
                    validation_steps= len(val_idx)//batch_size,
                    epochs = epochs,
#                     class_weight = class_weight,
                    validation_data = val_generator,
                    callbacks = [
                                 get_lr_callback(batch_size),
                                 get_ckpt()],
                   )
# Figure
plt.figure(figsize=(15,5))
plt.plot(np.arange(len(model.history.history['categorical_accuracy'])),model.history.history['categorical_accuracy'],'-o',label='Train categorical_accuracy',color='#ff7f0e')
plt.plot(np.arange(len(model.history.history['categorical_accuracy'])),model.history.history['val_categorical_accuracy'],'-o',label='Val categorical_accuracy',color='#1f77b4')
x = np.argmax( model.history.history['val_categorical_accuracy'] ); y = np.max( model.history.history['val_categorical_accuracy'] )
xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max categorical_accuracy\n%.2f'%y,size=14)
plt.ylabel('categorical_accuracy',size=14); plt.xlabel('Epoch',size=14)
plt.legend(loc=2)
plt2 = plt.gca().twinx()
plt2.plot(np.arange(len(model.history.history['loss'])),model.history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
plt2.plot(np.arange(len(model.history.history['loss'])),model.history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
x = np.argmin( model.history.history['val_loss'] ); y = np.min( model.history.history['val_loss'] )
ydist = plt.ylim()[1] - plt.ylim()[0]
plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
plt.ylabel('Loss',size=14)
plt.legend(loc=3)
plt.show()

# Predict Test Data

In [ ]:
test_generator = DataGenerator(path+'test_images/', samp_subm['image_id'], samp_subm['label'], 1, img_size, img_channel)
model.load_weights('best.h5')
predict = model.predict(test_generator, verbose=1)
samp_subm['label'] = predict.argmax(axis=1)

# Export Data

In [ ]:
samp_subm.to_csv('submission.csv', index=False)
samp_subm